# 1-S7: FastAPI 기초

ML 모델을 REST API로 서빙하기 위한 FastAPI를 학습합니다.

## 학습 목표
1. **FastAPI 기본** - 라우팅, 요청/응답
2. **Pydantic** - 데이터 검증 스키마
3. **비동기 처리** - async/await
4. **모델 서빙 패턴** - 로딩, 예측, 응답
5. **Docker 기초** - 컨테이너화

## 예상 시간
약 30분

In [2]:
%%time
# 필요한 패키지 설치 확인
!pip install fastapi uvicorn pydantic

import json
from typing import Optional, List, Dict, Any
from pydantic import BaseModel, Field

print("패키지 로드 완료!")

  Using cached pydantic-2.12.5-py3-none-any.whl.metadata (90 kB)
  Using cached starlette-0.50.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached annotated_doc-0.0.4-py3-none-any.whl.metadata (6.6 kB)
  Using cached click-8.3.1-py3-none-any.whl.metadata (2.6 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached typing_inspection-0.4.2-py3-none-any.whl.metadata (2.6 kB)
Using cached starlette-0.50.0-py3-none-any.whl (74 kB)
Using cached pydantic-2.12.5-py3-none-any.whl (463 kB)
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 2.0/2.0 MB 11.2 MB/s  0:00:00
Using cached annotated_doc-0.0.4-py3-none-any.whl (5.3 kB)
Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)
Using cached click-8.3.1-py3-none-any.whl (108 kB)
Using cached typing_inspection-0.4.2-py3-none-any.whl (14 kB)

   -------- ------------------------------- 2/9 [click]
   ---------------------- ----------------- 5/9 [

---
## 1. FastAPI란?

### 1-1. FastAPI 개념

**FastAPI**: Python 기반 고성능 웹 프레임워크

#### 왜 FastAPI인가?

| 프레임워크 | 특징 | ML 서빙 적합성 |
|-----------|------|---------------|
| **Flask** | 간단, 유연 | ⚠️ 동기 방식 |
| **Django** | 풀스택 | ❌ 과함 |
| **FastAPI** | 고성능, 자동 문서화 | ✅ 최적 |

#### FastAPI 장점

```
1. 성능: Flask 대비 3~5배 빠름 (비동기)
2. 타입 힌트: Pydantic으로 자동 검증
3. 자동 문서화: Swagger UI 자동 생성
4. 비동기: async/await 네이티브 지원
```

#### 기본 구조

```python
from fastapi import FastAPI

app = FastAPI()

@app.get("/")           # GET 요청
def read_root():
    return {"message": "Hello FDS!"}

@app.post("/predict")   # POST 요청
def predict(data: dict):
    return {"result": "fraud"}
```

### 1-2. HTTP 메서드

| 메서드 | 용도 | 예시 |
|--------|------|------|
| **GET** | 데이터 조회 | 거래 목록 조회 |
| **POST** | 데이터 생성/예측 | 사기 탐지 예측 |
| **PUT** | 데이터 수정 | 모델 설정 변경 |
| **DELETE** | 데이터 삭제 | 캐시 삭제 |

**FDS에서 주로 사용:**
- `POST /predict` - 사기 예측 요청
- `GET /health` - 서버 상태 확인

In [3]:
# 📚 FastAPI 기본 구조 (코드 예시)
# 실제 실행은 .py 파일에서!

fastapi_example = '''
from fastapi import FastAPI

app = FastAPI(
    title="FDS API",
    description="이상거래 탐지 API",
    version="1.0.0"
)

@app.get("/")
def root():
    """루트 엔드포인트"""
    return {"message": "FDS API is running!"}

@app.get("/health")
def health():
    """헬스 체크"""
    return {"status": "healthy"}
'''

print("FastAPI 기본 코드:")
print(fastapi_example)

FastAPI 기본 코드:

from fastapi import FastAPI

app = FastAPI(
    title="FDS API",
    description="이상거래 탐지 API",
    version="1.0.0"
)

@app.get("/")
def root():
    """루트 엔드포인트"""
    return {"message": "FDS API is running!"}

@app.get("/health")
def health():
    """헬스 체크"""
    return {"status": "healthy"}



---
## 2. Pydantic 스키마

### 2-1. Pydantic이란?

**Pydantic**: 데이터 검증 및 설정 관리 라이브러리

#### 왜 Pydantic?

```python
# ❌ 수동 검증 (지저분)
def predict(data: dict):
    if "amount" not in data:
        raise ValueError("amount 필수!")
    if not isinstance(data["amount"], (int, float)):
        raise ValueError("amount는 숫자!")
    if data["amount"] < 0:
        raise ValueError("amount는 양수!")
    ...

# ✅ Pydantic (깔끔)
class Transaction(BaseModel):
    amount: float = Field(gt=0)  # 양수만
    # 끝! 자동 검증됨
```

#### Pydantic 장점

| 기능 | 설명 |
|------|------|
| **자동 타입 변환** | "123" → 123 자동 변환 |
| **유효성 검증** | 필수 필드, 범위 체크 |
| **자동 문서화** | Swagger UI에 스키마 표시 |
| **JSON 직렬화** | dict ↔ JSON 자동 변환 |

In [8]:
# 📚 Pydantic 기본 사용법

from pydantic import BaseModel, Field
from typing import Optional, List

# 거래 데이터 스키마
class Transaction(BaseModel):
    """거래 정보 스키마"""
    transaction_id: str
    amount: float = Field(gt=0, description="거래 금액 (양수)")
    card_type: str = Field(default="credit", description="카드 종류")
    hour: int = Field(ge=0, le=23, description="거래 시간 (0-23)")
    is_weekend: bool = False

# 테스트
txn = Transaction(
    transaction_id="TXN001",
    amount=50000,
    hour=14
)

print("거래 객체:")
print(f"  ID: {txn.transaction_id}")
print(f"  금액: {txn.amount:,}원")
print(f"  카드: {txn.card_type}")
print(f"  시간: {txn.hour}시")
print(f"\nJSON 변환:")
print(txn.model_dump_json(indent=2))

거래 객체:
  ID: TXN001
  금액: 50,000.0원
  카드: credit
  시간: 14시

JSON 변환:
{
  "transaction_id": "TXN001",
  "amount": 50000.0,
  "card_type": "credit",
  "hour": 14,
  "is_weekend": false
}


In [9]:
# 📚 유효성 검증 예시

print("유효성 검증 테스트:")
print("-" * 40)

# 1. 정상 케이스
try:
    txn = Transaction(
        transaction_id="TXN001",
        amount=50000,
        hour=14
    )
    print("✅ 정상 데이터: 통과")
except Exception as e:
    print(f"❌ 에러: {e}")

# 2. 음수 금액
try:
    txn = Transaction(
        transaction_id="TXN002",
        amount=-1000,  # 음수!
        hour=14
    )
except Exception as e:
    print(f"✅ 음수 금액 거부됨")

# 3. 잘못된 시간
try:
    txn = Transaction(
        transaction_id="TXN003",
        amount=50000,
        hour=25  # 0-23 범위 벗어남!
    )
except Exception as e:
    print(f"✅ 잘못된 시간 거부됨")

# 4. 필수 필드 누락
try:
    txn = Transaction(
        amount=50000,
        hour=14
        # transaction_id 누락!
    )
except Exception as e:
    print(f"✅ 필수 필드 누락 거부됨")

print("-" * 40)
print("→ Pydantic이 자동으로 검증!")

유효성 검증 테스트:
----------------------------------------
✅ 정상 데이터: 통과
✅ 음수 금액 거부됨
✅ 잘못된 시간 거부됨
✅ 필수 필드 누락 거부됨
----------------------------------------
→ Pydantic이 자동으로 검증!


In [10]:
# 📚 FDS API 스키마 설계

from typing import List, Optional

# 요청 스키마
class PredictRequest(BaseModel):
    """예측 요청"""
    transaction_id: str
    amount: float = Field(gt=0)
    hour: int = Field(ge=0, le=23)
    card_type: str
    email_domain: Optional[str] = None

# 응답 스키마 - 피처 기여도
class FeatureFactor(BaseModel):
    """SHAP 피처 기여도"""
    feature: str
    feature_kr: str
    impact: float
    direction: str  # "increase" or "decrease"

# 응답 스키마 - 예측 결과
class PredictResponse(BaseModel):
    """예측 응답"""
    transaction_id: str
    fraud_probability: float = Field(ge=0, le=1)
    is_fraud: bool
    top_factors: List[FeatureFactor]
    explanation_text: str

# 테스트
response = PredictResponse(
    transaction_id="TXN001",
    fraud_probability=0.87,
    is_fraud=True,
    top_factors=[
        FeatureFactor(feature="amount", feature_kr="거래금액", impact=0.25, direction="increase"),
        FeatureFactor(feature="hour", feature_kr="거래시간", impact=0.18, direction="increase"),
    ],
    explanation_text="[사기 판단 근거]\n- 거래금액: 사기 위험 증가 (영향도 중간)"
)

print("예측 응답 스키마:")
print(response.model_dump_json(indent=2, ensure_ascii=False))

예측 응답 스키마:
{
  "transaction_id": "TXN001",
  "fraud_probability": 0.87,
  "is_fraud": true,
  "top_factors": [
    {
      "feature": "amount",
      "feature_kr": "거래금액",
      "impact": 0.25,
      "direction": "increase"
    },
    {
      "feature": "hour",
      "feature_kr": "거래시간",
      "impact": 0.18,
      "direction": "increase"
    }
  ],
  "explanation_text": "[사기 판단 근거]\n- 거래금액: 사기 위험 증가 (영향도 중간)"
}


---
## 3. 비동기 처리 (async/await)

### 3-1. 동기 vs 비동기

| 구분 | 동기 (Sync) | 비동기 (Async) |
|------|------------|---------------|
| 처리 방식 | 순차 실행 | 병렬 실행 |
| I/O 대기 | 블로킹 | 논블로킹 |
| 동시 요청 | 1개씩 | 여러 개 동시 |

#### 동기 방식 (Flask 스타일)

```
요청1 → [처리...] → 응답1
                         요청2 → [처리...] → 응답2
                                                  요청3 → ...
```

#### 비동기 방식 (FastAPI 스타일)

```
요청1 → [처리...]     → 응답1
요청2 →     [처리...] → 응답2
요청3 →       [처리...] → 응답3
```

**FDS에서 비동기가 중요한 이유:**
- 모델 추론 시간 동안 다른 요청 처리 가능
- 동시 요청 처리량 증가

In [11]:
# 📚 async/await 기본

import asyncio
import time

# 동기 함수
def sync_predict(data):
    time.sleep(0.1)  # 모델 추론 시간 시뮬레이션
    return {"result": "fraud"}

# 비동기 함수
async def async_predict(data):
    await asyncio.sleep(0.1)  # 비동기 대기
    return {"result": "fraud"}

# 동기 테스트
start = time.time()
for i in range(5):
    sync_predict({"id": i})
sync_time = time.time() - start

# 비동기 테스트
async def run_async():
    tasks = [async_predict({"id": i}) for i in range(5)]
    return await asyncio.gather(*tasks)

start = time.time()
asyncio.run(run_async())
async_time = time.time() - start

print(f"동기 처리 (5개 순차): {sync_time:.2f}초")
print(f"비동기 처리 (5개 동시): {async_time:.2f}초")
print(f"\n→ 비동기가 {sync_time/async_time:.1f}배 빠름!")

RuntimeError: asyncio.run() cannot be called from a running event loop

### 3-2. FastAPI에서 비동기

```python
# 동기 엔드포인트 (def)
@app.post("/predict")
def predict_sync(request: PredictRequest):
    result = model.predict(request)  # 블로킹
    return result

# 비동기 엔드포인트 (async def)
@app.post("/predict")
async def predict_async(request: PredictRequest):
    result = await model.predict(request)  # 논블로킹
    return result
```

**주의**: CPU 바운드 작업(모델 추론)은 async의 이점이 적음
- I/O 바운드 (DB 조회, 파일 읽기): async 효과 큼
- CPU 바운드 (모델 추론): 멀티프로세싱 필요

**FDS에서의 전략:**
```python
@app.post("/predict")
def predict(request: PredictRequest):  # def 사용
    # XGBoost/LSTM 추론은 CPU 바운드
    # FastAPI가 백그라운드 스레드풀에서 실행
    return model.predict(request)
```

---
## 4. 모델 서빙 패턴

### 4-1. 모델 로딩 패턴

**잘못된 방식:**
```python
@app.post("/predict")
def predict(request):
    model = load_model()  # ❌ 매 요청마다 로딩!
    return model.predict(request)
```

**올바른 방식:**
```python
# 앱 시작 시 한 번만 로딩
model = None

@app.on_event("startup")
def load_models():
    global model
    model = load_model()  # ✅ 시작 시 1회

@app.post("/predict")
def predict(request):
    return model.predict(request)  # 이미 로딩된 모델 사용
```

### 4-2. FDS API 구조

```
src/api/
├── main.py         # FastAPI 앱, 엔드포인트
├── schemas.py      # Pydantic 스키마
├── predictor.py    # 모델 로딩 및 예측 로직
└── __init__.py
```

In [12]:
# 📚 FDS API 전체 구조 예시

api_main_example = '''
# src/api/main.py
from fastapi import FastAPI
from .schemas import PredictRequest, PredictResponse
from .predictor import FDSPredictor

app = FastAPI(title="FDS API")

# 전역 변수로 모델 관리
predictor: FDSPredictor = None

@app.on_event("startup")
def startup():
    """앱 시작 시 모델 로딩"""
    global predictor
    predictor = FDSPredictor()
    predictor.load_models()

@app.get("/health")
def health():
    """헬스 체크"""
    return {"status": "healthy", "model_loaded": predictor is not None}

@app.post("/predict", response_model=PredictResponse)
def predict(request: PredictRequest):
    """사기 예측"""
    return predictor.predict(request)
'''

print("main.py 구조:")
print(api_main_example)

main.py 구조:

# src/api/main.py
from fastapi import FastAPI
from .schemas import PredictRequest, PredictResponse
from .predictor import FDSPredictor

app = FastAPI(title="FDS API")

# 전역 변수로 모델 관리
predictor: FDSPredictor = None

@app.on_event("startup")
def startup():
    """앱 시작 시 모델 로딩"""
    global predictor
    predictor = FDSPredictor()
    predictor.load_models()

@app.get("/health")
def health():
    """헬스 체크"""
    return {"status": "healthy", "model_loaded": predictor is not None}

@app.post("/predict", response_model=PredictResponse)
def predict(request: PredictRequest):
    """사기 예측"""
    return predictor.predict(request)



In [13]:
# 📚 Predictor 클래스 예시

predictor_example = '''
# src/api/predictor.py
import joblib
import numpy as np
from pathlib import Path
from .schemas import PredictRequest, PredictResponse, FeatureFactor

class FDSPredictor:
    """FDS 예측기"""
    
    def __init__(self):
        self.xgb_model = None
        self.explainer = None
        self.threshold = 0.5
    
    def load_models(self, models_dir: str = "models"):
        """모델 로딩"""
        models_path = Path(models_dir)
        
        # XGBoost 모델
        xgb_info = joblib.load(models_path / "xgb_model.joblib")
        self.xgb_model = xgb_info["model"]
        self.threshold = xgb_info["optimal_threshold"]
        
        # SHAP Explainer
        from src.explainer import ShapExplainer
        self.explainer = ShapExplainer(self.xgb_model, xgb_info["feature_names"])
    
    def predict(self, request: PredictRequest) -> PredictResponse:
        """예측 및 설명 생성"""
        # 1. 피처 추출
        features = self._extract_features(request)
        
        # 2. 예측
        prob = self.xgb_model.predict_proba(features)[0, 1]
        is_fraud = prob >= self.threshold
        
        # 3. SHAP 설명
        explanation = self.explainer.create_response(
            features, sample_idx=0, threshold=self.threshold
        )
        
        return PredictResponse(
            transaction_id=request.transaction_id,
            fraud_probability=float(prob),
            is_fraud=is_fraud,
            top_factors=explanation["top_factors"],
            explanation_text=explanation["explanation_text"]
        )
'''

print("predictor.py 구조:")
print(predictor_example)

predictor.py 구조:

# src/api/predictor.py
import joblib
import numpy as np
from pathlib import Path
from .schemas import PredictRequest, PredictResponse, FeatureFactor

class FDSPredictor:
    """FDS 예측기"""

    def __init__(self):
        self.xgb_model = None
        self.explainer = None
        self.threshold = 0.5

    def load_models(self, models_dir: str = "models"):
        """모델 로딩"""
        models_path = Path(models_dir)

        # XGBoost 모델
        xgb_info = joblib.load(models_path / "xgb_model.joblib")
        self.xgb_model = xgb_info["model"]
        self.threshold = xgb_info["optimal_threshold"]

        # SHAP Explainer
        from src.explainer import ShapExplainer
        self.explainer = ShapExplainer(self.xgb_model, xgb_info["feature_names"])

    def predict(self, request: PredictRequest) -> PredictResponse:
        """예측 및 설명 생성"""
        # 1. 피처 추출
        features = self._extract_features(request)

        # 2. 예측
        prob = self.xgb_model.predict_proba(

---
## 5. Docker 기초

### 5-1. Docker란?

**Docker**: 애플리케이션을 컨테이너로 패키징하여 어디서든 실행

#### 왜 Docker?

```
❌ 문제: "내 컴퓨터에서는 됐는데..."
   - Python 버전 차이
   - 패키지 버전 차이
   - OS 차이

✅ Docker: 환경 자체를 패키징
   - 개발 환경 = 운영 환경
   - 어디서든 동일하게 실행
```

### 5-2. Dockerfile

```dockerfile
# Dockerfile
FROM python:3.11-slim

WORKDIR /app

# 의존성 설치
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

# 코드 복사
COPY . .

# 포트 노출
EXPOSE 8000

# 실행 명령
CMD ["uvicorn", "src.api.main:app", "--host", "0.0.0.0", "--port", "8000"]
```

### 5-3. Docker Compose

```yaml
# docker-compose.yml
version: '3.8'

services:
  fds-api:
    build: .
    ports:
      - "8000:8000"
    volumes:
      - ./models:/app/models
    environment:
      - MODEL_PATH=/app/models
```

### 5-4. Docker 명령어

```bash
# 이미지 빌드
docker build -t fds-api .

# 컨테이너 실행
docker run -p 8000:8000 fds-api

# docker compose로 실행 (권장)
docker compose up -d

# 로그 확인
docker compose logs -f

# 종료
docker compose down
```

---
## 면접 Q&A

### Q: "왜 FastAPI를 선택했나요?"

> "세 가지 이유입니다. 첫째, 성능이 Flask 대비 3~5배 빠릅니다.
> 둘째, Pydantic으로 자동 타입 검증과 문서화가 됩니다.
> 셋째, 비동기 지원이 네이티브라 동시 요청 처리에 유리합니다."

### Q: "API 응답 시간은 얼마나 걸리나요?"

> "XGBoost 예측 약 10ms, SHAP 계산 약 100ms로 총 약 120ms입니다.
> 목표였던 200ms 이하를 달성했습니다.
> 배치 예측을 지원하면 처리량을 더 높일 수 있습니다."

### Q: "모델 업데이트는 어떻게 하나요?"

> "현재는 Docker 이미지 재빌드 방식입니다.
> 모델 파일을 볼륨 마운트하면 이미지 재빌드 없이 교체 가능합니다.
> Phase 2에서 MLflow로 모델 버전 관리를 추가할 예정입니다."

### Q: "동시 요청이 많아지면 어떻게 처리하나요?"

> "Uvicorn 워커 수를 늘려서 스케일업합니다.
> 더 많은 부하는 Kubernetes로 스케일아웃합니다.
> 현재 단일 서버로 초당 50~100 요청 처리 가능합니다."

---
## 최종 체크포인트

In [ ]:
print("=" * 60)
print("  1-S7 완료: FastAPI 기초")
print("=" * 60)
print()
print("배운 것:")
print()
print("1. FastAPI 기본")
print("   - 고성능 웹 프레임워크")
print("   - @app.get(), @app.post() 데코레이터")
print("   - 자동 Swagger 문서화")
print()
print("2. Pydantic 스키마")
print("   - BaseModel로 데이터 검증")
print("   - Field()로 제약조건 설정")
print("   - 자동 JSON 변환")
print()
print("3. 비동기 처리")
print("   - async/await 문법")
print("   - I/O 바운드 작업에 효과적")
print("   - CPU 바운드는 def 사용")
print()
print("4. 모델 서빙 패턴")
print("   - 앱 시작 시 모델 로딩 (@on_event)")
print("   - Predictor 클래스로 캡슐화")
print("   - schemas.py, predictor.py 분리")
print()
print("5. Docker 기초")
print("   - Dockerfile로 환경 패키징")
print("   - docker compose up으로 실행")
print()
print("=" * 60)
print("다음: src/api/ 구현")
print("=" * 60)